# Tratamento de Planilha para o Relatório de Transparência



##Bibliotecas & Funções

In [ ]:
import pandas as pd
import numpy as np
import io

In [ ]:
#conecta com o Google Drive para permitir acesso a pasta em que estão armazenados os arquivos csv.
from google.colab import drive
drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive
Mounted at /content/drive


In [ ]:
def tratamento_planilhas_antigas(planilha_antiga, planilha_atual):
  planilha_antiga['Identificação'] = planilha_antiga['Identificação'].str.upper() #
  planilha_antiga.rename(columns={' Valor':'Valor'}, inplace=True)
  i=0
  for item in planilha_antiga['Valor'].values:
    planilha_antiga['Valor'][i] = planilha_antiga['Valor'][i].replace('R$', '')
    planilha_antiga['Valor'][i] = planilha_antiga['Valor'][i].replace(' ', '')
    planilha_antiga['Valor'][i] = planilha_antiga['Valor'][i].replace(',', '')
    i+= 1
  planilha_antiga['Valor']=planilha_antiga['Valor'].astype(float)
  #adicionar categorias com base em planilha antiga
  i=0
  tam1 = len(planilha_atual.index)
  tam2 = len(planilha_antiga.index)
  planilha_antiga['Identificação'] = planilha_antiga['Identificação'].str.upper()
  for i in range(tam1):
    for j in range(tam2):
      if ( (planilha_atual['Identificação'][i] == planilha_antiga['Identificação'][j]) and (planilha_atual['Valor'][i] == planilha_antiga['Valor'][j])):
        planilha_atual['Categoria'][i] = planilha_antiga['Categoria'][j]
      j += 1
    i = i+1
  return planilha_atual

## Importação de Planilhas

\  Primeiro, importamos a planilha do [Projeto Interno] ( a essencial) e de meses anteriores.



 * Dica: dar preferência a planilhas anuais/semestrais. Colocar tudo numa pasta do Google drive.

In [ ]:
projetointerno = pd.read_csv("drive/MyDrive/planilhas para tratamento/projetointerno.csv")
#2022 -> transforme num arquivo csv único.
AntigaJanAgo = pd.read_csv('drive/MyDrive/planilhas para tratamento/jan-ago.csv')
Antiga0922 = pd.read_csv('drive/MyDrive/planilhas para tratamento/09.csv')
#Algumas planilhas antigas necessitam de ajustes, como o caso da planilha de Setembro de 2022.
Antiga0922.rename(columns={'Identificaçao': 'Identificação', 'Transaçao': 'Transação', 'Tipo Transaçao': 'Tipo Transação', 'Saida':'Saída'}, inplace = True)
Antiga1022 = pd.read_csv('drive/MyDrive/planilhas para tratamento/planilhatratada.csv')
#2023.1

#2023.2 -> adicione aqui os arquivos csv de cada mês

Em seguida, importamos a planilha que queremos tratar. Modifique o 'file.csv' para o nome do arquivo csv que você quer importar.

In [ ]:
from google.colab import files
uploaded = files.upload()
planilha_atual = pd.read_csv(io.BytesIO(uploaded['cora_agosto.csv']))

Saving cora_agosto.csv to cora_agosto.csv


## Tratamento Básico da Planilha Atual

In [ ]:
planilha_atual.insert(3, "Banco", "nome do banco", True) #Cria coluna na posição 3, chama de Banco e nomeia todos os elementos como "nome do banco"
planilha_atual = planilha_atual.replace({'Transf Pix enviada':'PIX', 'Pgto QR Code Pix':'PIX', 'Boleto pago':'BOLETO'}) #renomeia todos os elementos do dataframe para as respectivas alterações
planilha_atual = planilha_atual.assign(Categoria='NaN') #cria coluna de categoria
planilha_atual['Identificação'] = planilha_atual['Identificação'].str.upper() #deixa os nomes na coluna identificação em maiúsculo
planilha_atual = planilha_atual.assign(Entrada='NaN') #ceia coluna de entrada
planilha_atual = planilha_atual.assign(Saída='NaN') #cria coluna de saída
planilha_atual = planilha_atual.sort_values('Data') #ordena os valores da planilha por data.

somatorio = 0
#O laço percorre toda a planilha e vai somando os valores dos pagamentos e armazenando na variável somatório.
for index, row in planilha_atual.iterrows():
  somatorio += row['Valor']
  planilha_atual.at[index, 'Valor acumulado'] = somatorio #o valor atual do somatório em cada iteração do laço é sempre adicionado na célula atual de "valor acumulado"
  if row['Valor'] > 0: #se os valores forem positivos, são entradas
    planilha_atual.at[index, 'Entrada'] = row['Valor']
  elif row['Valor'] < 0: #se forem negativos, são saídas
    planilha_atual.at[index, 'Saída'] = row['Valor']
    #Adiciona categorias frequentes com base na identificação do pagamento/valor.
  if row['Identificação'] == 'DISTRIBUIDORA E LOGISTICA DE PE IMPORTACAO E EXPOR':
    planilha_atual.at[index, 'Categoria'] = 'Custos do CITi'
  if row['Identificação'] == 'PREFEITURA DA CIDADE DO R' or row['Identificação'] =='DARF - SEM CÓDIGO DE BARRAS' or row['Identificação'] =='DARF - SEM CODIGO DE BARRAS':
    planilha_atual.at[index, 'Categoria'] = 'Impostos'
    planilha_atual.at[index, 'Identificação'] = 'PREFEITURA DA CIDADE DO RECIFE'
  if row['Identificação'] =='YAPAY PAGAMENTOS ONLINE LTDA' or row['Identificação'] =='PJBANK PAGAMENTOS S.A' or row['Identificação'] =='ELEVE TECNOLOGIA I LTDA ME' or row['Identificação'] =='Pulses Servicos Digitais S A' :
    planilha_atual.at[index, 'Categoria'] = 'Plataformas'
  if ((row['Identificação'] =='CENTRO INTEGRADO DE TECNOLOGIA DA INFORM')and (row['Valor'] >= -3000.00)):
    planilha_atual.at[index, 'Categoria'] = 'ContaSimples'
  if (row['Identificação'] =='planilha_atual SCD'):
    planilha_atual.at[index, 'Categoria'] = 'Fatura planilha_atual'
  if (row['Identificação'] =='BANCO BS2 SA'):
    planilha_atual.at[index, 'Categoria'] = 'Recebimento de cliente'
  if (row['Identificação'] =='PONTUALLY ASSESSORIA E PERICIA CONTABIL' or row['Identificação'] =='KOTTLER COMUNICACOES LTDA' or row['Identificação'] =='ANA KARLA BARBOSA DA SILVA' or row['Identificação'] =='ASAAS GESTAO FINANCEIRA S A' ):
    planilha_atual.at[index, 'Categoria'] = 'Terceirização'

## Tratamento básico da planilha do projetointerno
É importante que o nome das colunas sejam compatíveis entre a planilha_atual e a do projetointerno.

In [ ]:
#transformação em lista
qtd_pessoas = 9
projetointerno.rename(columns={'Nome do Titular':'Identificação'}, inplace=True) #renomeia coluna de Nome do Titular para Identificação.
projetointerno['Identificação'] = projetointerno['Identificação'].str.upper() #deixa todos os nomes de identificação maiúsculos
lista = projetointerno['Identificação'].tolist() #transforma a coluna de identificação numa lista
for i in range(0, qtd_pessoas):
      lista[i] = lista[i].strip() #tira espaços em branco no final da lista (verifique se realmente precisa na desse ano)

In [ ]:
#adiciona categoria "projetointerno"
#verifica se os valores e os nomes das pessoas são compatíveis.
for i, item in zip(planilha_atual.index, planilha_atual['Identificação']):
  for j in range(len(lista)):
    if (item == lista[j] and (planilha_atual.at[i, 'Valor'] == -200.00 or planilha_atual.at[i, 'Valor'] == -300.00 or planilha_atual.at[i, 'Valor'] == -400.00)):
      planilha_atual.at[i, 'Categoria'] = 'projetointerno'

## Tratamento das Planilhas Antigas & Acréscimo de Categorias na Planilha Atual

Utilize a função do início deste código para todas as planilhas anteriores que você importou anteriormente.

In [ ]:
planilha_atual = tratamento_planilhas_antigas(AntigaJanAgo, planilha_atual)

<ipython-input-3-a61b3fe92732>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cora['Categoria'][i] = planilha_antiga['Categoria'][j]


In [ ]:
planilha_atual = tratamento_planilhas_antigas(Antiga0922, planilha_atual)

<ipython-input-3-a61b3fe92732>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cora['Categoria'][i] = planilha_antiga['Categoria'][j]


In [ ]:
planilha_atual = tratamento_planilhas_antigas(Antiga1022, planilha_atual)

<ipython-input-3-a61b3fe92732>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cora['Categoria'][i] = planilha_antiga['Categoria'][j]


## Transforma o csv em arquivo Excel
Não esqueça de modificar o nome do arquivo para o mês do qual você está fazendo o relatório.

In [ ]:
arquivo = 'agosto_tratado.xlsx'
planilha_atual.to_excel(arquivo)